In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.0.0")
@file:DependsOn("io.ktor:ktor-client-cio:2.0.0")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.0.0")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-api:1.0.0")

import io.ktor.client.*
import io.ktor.client.engine.cio.*
import io.ktor.client.plugins.contentnegotiation.*
import io.ktor.client.request.*
import io.ktor.client.request.forms.*
import io.ktor.serialization.kotlinx.json.*
import kotlinx.serialization.Serializable
import kotlinx.serialization.json.Json
import ro.jf.funds.importer.sdk.ImportSdk

val httpClient = HttpClient(CIO) {
    install(ContentNegotiation) {
        json(Json {
            prettyPrint = true
            isLenient = true
        })
    }
}

val importService = ImportSdk(httpClient)

In [3]:
import kotlinx.coroutines.runBlocking
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import java.util.UUID.randomUUID

val userId = randomUUID()
val csvFile = File("../../data/playground/csv-v2.csv")
val importConfiguration = ImportConfigurationTO(
    keys = KeysTO(
        accountName = "cont",
        currency = "valuta",
        amount = "suma",
        date = "data"
    ),
    formatting = FormattingTO(
        csvDelimiter = ";",
        dateTimeFormat = "yyyy-MM-dd'T'HH:mm",
        locale = Locale.FRENCH
    ),
    transactionIdentification = TransactionIdentificationTO(
        conditions = listOf(
            TransactionConditionTO.ColumnEquality("tranzactie"),
            TransactionConditionTO.ColumnConstant("tip", "TRANSFER"),
            TransactionConditionTO.NumericalAbsoluteEquality("suma")
        )
    )
)
val response = runBlocking { importService.import(userId, csvFile, importConfiguration) }
response

ImportResponse(response=Imported in service)